This is generic introduction on how to use the module written. The module can be imported as

In [1]:
import qcomp as Q
import numpy as np

# QBits

QBits are pretty much dumb on their own. Only thing that needed caution is making sure it is normalized. This is done automatically so there is no need to check

In [2]:
Q.qbit.QBit([-2,20]).state

array([-0.09950373+0.j,  0.99503726+0.j], dtype=complex64)

There are four useful pre-defined bits that are useful when preparing registers |0>,|1>,|0>+|1>,|0>-|1>. As with anything, you can usually call _help_ to get information.

In [3]:
help(Q.qbit)

Help on module qcomp.qbit in qcomp:

NAME
    qcomp.qbit

DESCRIPTION
    qbit module contains class QBit, and constants
    ZERO, ONE which represent 1-qbit |0> and |1> states

CLASSES
    builtins.object
        QBit
    
    class QBit(builtins.object)
     |  QBit is represented as 2D complex vector
     |  
     |  Methods defined here:
     |  
     |  __init__(self, state)
     |      Instance variables
     |      -----------------
     |      state: numpy array (size 2)
     |      
     |      Methods
     |      -----------------
     |      normalize
     |  
     |  normalize(self)
     |      Set the magnitude of state vector to 1, keeps overall probability equal to 1
     |      Does not return anything
     |  
     |  ----------------------------------------------------------------------
     |  Data descriptors defined here:
     |  
     |  __dict__
     |      dictionary for instance variables (if defined)
     |  
     |  __weakref__
     |      list of weak refere

to import predefined bits we want

In [4]:
from qcomp.qbit import ZERO, ONE, PLUS, MINUS
print(PLUS.state)

[0.70710677+0.j 0.70710677+0.j]


# Quantum Register

Quantum register uses kron product to join individual qbits. After that it is manipulated almost exclusively by using gates (next section). It is currently impossible to _add_, or _join_ registers to form new one, because it sounds weird to me that it is even possible. 

Main friend here is function _mk_reg_ which creates register given list of bits. 

Other useful methods to know about:

- copy: create copy of register
- swap(i,j): swap two bits in register
- gather(bits): gather necessary bits together for applying gates **NOT IMPLEMENTED**
- scatter(bits): reverse of gather **NOT IMPLEMENTED**
- get_qbit(i): get state of ith (single) qbit (in string form). 

In [5]:
from qcomp.qregister import mk_reg

In [6]:
Q00 = mk_reg([ZERO,ZERO])
QMM = mk_reg([MINUS, MINUS])

additionally we can use len and print functions on register. They work as expected

In [7]:
print(Q00)

1.00+0.00j 	 |00> 
0.00+0.00j 	 |01> 
0.00+0.00j 	 |10> 
0.00+0.00j 	 |11> 



In [8]:
len(QMM)

2

In [9]:
print(QMM)

0.50+0.00j 	 |00> 
-0.50+0.00j 	 |01> 
-0.50+0.00j 	 |10> 
0.50-0.00j 	 |11> 



# GATES

Main heart of Quantum algorithms is the use of gates. Generic Gate only needs to know 2 things, how many qbits it is acting on and how it actually acts. For now, everything is pretty much matrix multiplication all over, however things can be changed to create better performance

## Matrix gate

This is simply gate constructed by passing matrix. Generally not used, however is last resort and will theoretically always work

In [10]:
from qcomp.gate import MGate

In [11]:
inot = MGate(np.array([
    [0,1,0,0],
    [1,0,0,0],
    [0,0,0,1],
    [0,0,1,0]
]), 2) # this simply reverses second bit
inot.apply(Q00)

0.00+0.00j 	 |00> 
1.00+0.00j 	 |01> 
0.00+0.00j 	 |10> 
0.00+0.00j 	 |11> 

## Single QBit gates

These are Hadamard, ID, PhaseShift and NOT gates. Important thing to always be cautious is these act on register which has only one bit, not on the bit itself. Just call apply method.

In [12]:
from qcomp.gate import PShiftGate, NOT, H, I

In [13]:
# help is always our friend
help(PShiftGate)

Help on class PShiftGate in module qcomp.gate:

class PShiftGate(MGate)
 |  Phase Shift Gate (on single qbit)
 |  
 |  Method resolution order:
 |      PShiftGate
 |      MGate
 |      Gate
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, phi)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from MGate:
 |  
 |  apply(self, qreg)
 |      Apply the gate to the given register 
 |      Parameters
 |      -----------
 |      **qreg** : QReg instance 
 |      
 |      Returns
 |      ----------
 |      new_qreg : QReg object created after transformation
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from Gate:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [14]:
QPLUS = mk_reg([PLUS])

In [15]:
H.apply(QPLUS)

1.00+0.00j 	 |0> 
0.00+0.00j 	 |1> 

In [16]:
# not a good idea to do it this way,  but hey it works
NOT.apply(H.apply(QPLUS))

0.00+0.00j 	 |0> 
1.00+0.00j 	 |1> 

In [17]:
pishift = PShiftGate(np.pi/2)
pishift.apply(mk_reg([MINUS]))

0.71+0.00j 	 |0> 
-0.00-0.71j 	 |1> 

## VChain

What if we have 5 qbit system and we want to apply hadamard on 2nd and 3rd. Unfortunately we cannot extract those bits and apply Hadamard on each then put them back. Since they are probably entangled with other bits, this would destry the state of register. Instead we need to construc a gate that applies to the whole system. This is what VChain is used for. Simply pass the list of gates (starting from 0th qbit) that needs to be applied. If particular bit does need any change just pass Identity gate. The gates will be broadcasted into bigger gate (by kron product) and be ready to applied to anything

**! NOTE** this one is probably the main performance bottleneck. Needs some clever tweaks

In [18]:
from qcomp.gate import VChain

In [19]:
Q010M = mk_reg([ONE, ONE, ZERO, ONE])

In [20]:
bigguy = VChain([I,NOT,H,NOT]) # hadamard on 2nd, not on 1st
bigguy.apply(Q010M)

0.00+0.00j 	 |0000> 
0.00+0.00j 	 |0001> 
0.00+0.00j 	 |0010> 
0.00+0.00j 	 |0011> 
0.00+0.00j 	 |0100> 
0.00+0.00j 	 |0101> 
0.00+0.00j 	 |0110> 
0.00+0.00j 	 |0111> 
0.71+0.00j 	 |1000> 
0.00+0.00j 	 |1001> 
0.71+0.00j 	 |1010> 
0.00+0.00j 	 |1011> 
0.00+0.00j 	 |1100> 
0.00+0.00j 	 |1101> 
0.00+0.00j 	 |1110> 
0.00+0.00j 	 |1111> 

## HChain

When we wanna apply one gate after another, we can simply do
::    

    >>> q1 = G1.apply(q0)
    >>> q2 = G2.apply(q1)

and so on. HChain automates this by simply storing list of gates and applying them successively in one go

In [21]:
from qcomp.gate import HChain

secondbigguy = VChain([I,I,I,H])
biggest_guy = HChain([bigguy, secondbigguy])
biggest_guy.apply(Q010M)

0.00+0.00j 	 |0000> 
0.00+0.00j 	 |0001> 
0.00+0.00j 	 |0010> 
0.00+0.00j 	 |0011> 
0.00+0.00j 	 |0100> 
0.00+0.00j 	 |0101> 
0.00+0.00j 	 |0110> 
0.00+0.00j 	 |0111> 
0.50+0.00j 	 |1000> 
0.50+0.00j 	 |1001> 
0.50+0.00j 	 |1010> 
0.50+0.00j 	 |1011> 
0.00+0.00j 	 |1100> 
0.00+0.00j 	 |1101> 
0.00+0.00j 	 |1110> 
0.00+0.00j 	 |1111> 

## Two-qubit gates

This part is where things get tricky. Because acting on two gates can create entangled state or it can act on entangled state to begin with things are not as flexible. However, they still have same fingerprint as generic gate and are fairly easy to use. 

2-QBit gates are simple MGates with 4x4 matrix size. 

### SGate

This is the real motherfucker. Normally, MGate can only act on neighbouring gates (say, 3rd and 4th). Here we use simple trick

1. swap bits we wanna work on with last two bits
2. Apply Identity gate to everything except last two (IDEA->There can be performance boost here)
3. Apply MGate to last two
4. Put everything back

This way we can entangle any two qbits we want. Performace pay-off is swapping itself, however, I imagine it is cheaper then 'kron'ing gates. 

The irritating part with this one is that, it cannot be VChained because this itself is VChain on its own (which works a little bit different). So this gate needs to act on **entire** register.

#### controlled gates

However, I have not come across anything that have actually needed SGate directly. More useful ones are its descendents, controlled-NOT (CNOT), controlled-PhaseShif(CPSGate). They all have form 
  1 0 0 0
  0 1 0 0
  0 0 a b
  0 0 c d
This means you only need to know a,b,c,d to use the gates. In case of **CNOT** it same as NOT gate, for **CPSGate** it is same as phase shift gate. You need to pass extra 3 arguments, position of control bit, position of acting bit and size of register

In [22]:
from qcomp.gate import CNOT, CPSGate

In [23]:
help(CNOT)

Help on class CNOT in module qcomp.gate:

class CNOT(CGate)
 |  Controlled gate. This one will act on the second bit (in some predefined way) when control bit is 0. Otherwise it stays intact. Controlled gates have matrix in the from
 |  [1 0 0  0]
 |  [0 1 0 0]
 |  [0 0 a b]
 |  [0 0 c d]
 |  where a,b,c,d are transformation matrix that will be applied (when cbit is 1). These are SGate, so cannot be V-Chained
 |  
 |  Method resolution order:
 |      CNOT
 |      CGate
 |      SGate
 |      Gate
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, cbit, abit, qreg_size)
 |      Make a gate that has defined matrix form, which will entangle bits specified rather than neighbours.
 |      
 |      Parameters:
 |      matrix.: matrix form of gate (4x4 expected as it will act on 2 bits)
 |      control_bit: first bit it acts on (usually used as control)
 |      act_bit: second bit it acts on (usually bit that will change when control bit is 0)
 |  
 |  -------------

In [24]:
Q01P = mk_reg([ZERO,ONE,PLUS])
Q01P

0.00+0.00j 	 |000> 
0.00+0.00j 	 |001> 
0.71+0.00j 	 |010> 
0.71+0.00j 	 |011> 
0.00+0.00j 	 |100> 
0.00+0.00j 	 |101> 
0.00+0.00j 	 |110> 
0.00+0.00j 	 |111> 

In [25]:
cnot20 = CNOT(2,0,3) # switch switch 0th bit, controlled by 2nd bit
cnot20.apply(Q01P)

0.00+0.00j 	 |000> 
0.71+0.00j 	 |001> 
0.00+0.00j 	 |010> 
0.00+0.00j 	 |011> 
0.00+0.00j 	 |100> 
0.71+0.00j 	 |101> 
0.00+0.00j 	 |110> 
0.00+0.00j 	 |111> 

half of 0th bits are 0, other half is 1. Just as expected

In [26]:
cps01 = CPSGate(np.pi/2,1,2,3) # 1st bit is control, 0th will be acted upon
cps01.apply(Q01P)

0.00+0.00j 	 |000> 
0.71+0.00j 	 |001> 
0.00+0.00j 	 |010> 
0.00+0.00j 	 |011> 
0.00+0.00j 	 |100> 
0.00+0.71j 	 |101> 
0.00+0.00j 	 |110> 
0.00+0.00j 	 |111> 

**THE ANSWER IS NOT RIGHT, FOR SOME REASON IT HAS APPLIED REVERSING TO BITS AS WELL AS PHASE SHIFT. NEEDS FURTHER INSPECTION**

### CCNOT gate

This one is extension of CNOT gate where two bits are used as control (when both is 1, applies not to acting bit). Cool thing about this is by apply not gate to controls we can switch types of control. For example, if I apply not gate to first bit, then gate acts when controls are 01 (rather than 11). This is amazing as now, we can implement classic logic gates easily as well.

In [27]:
Q010 = mk_reg([ZERO,ONE,ZERO])

In [28]:
from qcomp.gate import CCNOT
help(CCNOT)

Help on class CCNOT in module qcomp.gate:

class CCNOT(CCGate)
 |  Similar to CGate except two bits are used for control. When both is one transformation is applied, otherwise nothing is done
 |  
 |  Method resolution order:
 |      CCNOT
 |      CCGate
 |      Gate
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, cbits, abit, qreg_size)
 |      by defualt prepares 3 qbit gate that uses first 2 as control, acts on 3rd
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from CCGate:
 |  
 |  apply(self, qreg)
 |      Apply the gate to the given register 
 |      Parameters
 |      -----------
 |      **qreg** : QReg instance 
 |      
 |      Returns
 |      ----------
 |      new_qreg : QReg object created after transformation
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from Gate:
 |  
 |  __dict__
 |      dictionary for instance variables 

In [ ]:
ccnot = CCNOT([0,1], 2, 3)

In [39]:
ccnot.apply(Q010) # nothing should apply as one of controls is 0

0.00+0.00j 	 |000> 
0.00+0.00j 	 |001> 
1.00+0.00j 	 |010> 
0.00+0.00j 	 |011> 
0.00+0.00j 	 |100> 
0.00+0.00j 	 |101> 
0.00+0.00j 	 |110> 
0.00+0.00j 	 |111> 

In [45]:
notccnot = HChain([
    VChain([NOT,I,I]),
    ccnot,
    VChain([NOT,I,I]) # do not forget to flip things back
])
notccnot.apply(Q010) # now we defined ou control to 01 so last bit should be flipped

AttributeError: 'CCNOT' object has no attribute 'qreg_size'